In [ ]:
--Bronze Layer
-- create a materialized view over diagnosis_mapping raw (static) table
CREATE LIVE TABLE diagnosis_mapping (
  CONSTRAINT diagnosis_code_not_null EXPECT (diagnosis_code IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT diagnosis_desc_not_null EXPECT (diagnosis_description IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Bronze Table for Diagnosis Mapping"
TBLPROPERTIES (quality = "bronze")
AS
SELECT 
  CAST(diagnosis_code AS STRING) AS diagnosis_code,
  CAST(diagnosis_description AS STRING) AS diagnosis_description
FROM 
  healthcare_dlt_medallion.default.raw_diagnosis_mapping;

In [ ]:
--Bronze Layer
--streaming source: delta tables (raw_patients_daily)
CREATE OR REFRESH STREAMING TABLE daily_patients (
  CONSTRAINT patient_id_not_null EXPECT (patient_id IS NOT NULL) ON VIOLATION DROP ROW,
  --business rules for other fields
  CONSTRAINT required_fields EXPECT (name IS NOT NULL AND age IS NOT NULL AND gender IS NOT NULL AND address IS NOT NULL AND contact_number IS NOT NULL AND admission_date IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Bronze Table to ingest new data for daily patient admissions"
TBLPROPERTIES (quality = "bronze")
AS 
SELECT
  CAST(patient_id AS STRING) AS patient_id,
  CAST(name AS STRING) AS name,
  CAST(age AS INT) AS age,
  CAST(gender AS STRING) AS gender,
  CAST(address AS STRING) AS address,
  CAST(contact_number AS STRING) AS contact_number,
  CAST(admission_date AS DATE) AS admission_date,
  CAST(diagnosis_code AS STRING) AS diagnosis_code 
FROM
  STREAM(healthcare_dlt_medallion.default.raw_patients_daily);

In [ ]:
--Silver layer 
--enrich daily patient data with diagnosis descriptions
CREATE OR REFRESH STREAMING TABLE processed_patients_data (
  CONSTRAINT has_diagnosis EXPECT (diagnosis_description IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Silver Table to enrich daily patient data with diagnosis descriptions"
TBLPROPERTIES (quality = "silver")
AS 
SELECT 
  dp.patient_id,
  dp.name,
  dp.age,
  dp.gender,
  dp.address,
  dp.contact_number,
  dp.admission_date,
  dm.diagnosis_description AS diagnosis_description
FROM 
  STREAM(live.daily_patients) dp
LEFT JOIN --left join to preserve patient data if diagnosis is not populated
  live.diagnosis_mapping dm
ON dp.diagnosis_code = dm.diagnosis_code;

In [ ]:
--Gold layer 
--This table creates aggregated analytics for daily patient admissions by diagnosis
CREATE LIVE TABLE patient_stats_by_admission_date
COMMENT "Gold Table to create aggregated analytics for daily patient admissions by diagnosis"
TBLPROPERTIES (quality = "gold")
AS
SELECT
  admission_date,
  diagnosis_description,
  COUNT(*) AS num_patients,
  AVG(age) AS avg_age
FROM
  live.processed_patients_data
GROUP BY admission_date, diagnosis_description;

In [ ]:
--Gold Layer
--This table creates comprehensive analytics for each diagnosis type
CREATE LIVE TABLE patient_stats_by_diagnosis
COMMENT "Gold table with comprehensive patient statistics by diagnosis"
TBLPROPERTIES (quality = "gold")
AS
SELECT 
  diagnosis_description,
  COUNT(patient_id) AS num_patients,
  AVG(age) AS age,
  MIN(age) AS min_age,
  MAX(age) AS max_age,
  COUNT(distinct gender) AS unique_gender_count
FROM live.processed_patients_data
GROUP BY diagnosis_description;

In [ ]:
%sql
--Gold Layer
--This table creates demographic analytics grouped by patient gender
CREATE LIVE TABLE patient_stats_by_gender 
COMMENT "Gold table with demographic patient statistics by gender"
TBLPROPERTIES (quality = "gold")
AS
SELECT
  gender,
  COUNT(patient_id) AS num_patients,
  AVG(age) AS age,
  MIN(age) AS min_age,
  MAX(age) AS max_age,
  COUNT(distinct diagnosis_description) AS unique_diagnosis_count
FROM live.processed_patients_data
GROUP BY gender;